In [1388]:
#############################################################################
#
# Global parameters
#
#####################

target_idx=0                                        #Index of Target variable
cross_val=0                                         #Control Switch for CV
norm_target=0                                       #Normalize target switch
norm_features=0                                     #Normalize features switch
prebinned = 1                                       #Control if the target are classes
binning=1                                           #Control Switch for Bin Target
bin_cnt=3                                           #If bin target, this sets number of classes
feat_start=1                                        #Start column of features
k_cnt=5                                             #Number of 'Top k' best ranked features to select, only applies for fs_types 1 and 3
lv_filter=1                                         #Control switch for low variance filter on features
feat_select= 0                                      #Control Switch for Feature Selection
fs_type=3                                           #Feature Selection type (1=Stepwise Backwards Removal, 2=Wrapper Select, 3=Univariate Selection)


#Set global model parameters
rand_st=0                                           #Set Random State variable for randomizing splits on runs
feat_eng = 1                             #IF 1, run all types of feature engineering and output results

In [1389]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [1390]:
data = pd.read_csv('categorical_dataset_ML1.csv').iloc[:, 1:]
data.head()

,G3,school,sex,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,age,absences
0,0,0,0,1,0,0,4,4,0,4,...,0,0,3,2,3,0,0,2,18,6
1,0,0,0,1,0,1,1,1,0,2,...,1,0,4,2,2,0,0,2,17,4
2,0,0,0,1,1,1,1,1,0,2,...,1,0,3,2,1,1,2,2,15,10
3,1,0,0,1,0,1,4,2,1,3,...,1,1,2,1,1,0,0,4,15,2
4,0,0,0,1,0,1,3,3,2,2,...,0,0,3,2,1,0,1,4,16,4


# Feature Engineering

    - Walc and Dalc into 1
    - merge 3 types of support
    - socializing (ratio between freetime and going out)
    
    
    - Medu and Mjob very related, maybe only keep Medu?
    - Medu and Fedu very related. Keep only that of guardian?
        - Or just keep Medu
    - Make and average parent education
    - Keep only guardian edu / job

In [1392]:
########## Average Alcohol ###########
data = pd.read_csv('categorical_dataset_ML1.csv').iloc[:, 1:]

Walc = data.pop('Walc')
Dalc = data.pop('Dalc')

Avg_alcohol = pd.DataFrame({'Alcohol': (Walc + Dalc) / 2})
print(Avg_alcohol)

data = pd.concat( [data, Avg_alcohol], axis = 1)

# data.to_csv('modelling_data.csv')

In [1378]:
# ######## Merge Support types --> # Support ###########
# # For None -> 0 ; # For family -> 1 ; # For school -> 2 ; # For paid -> 3 ; # For +1 type of support ->4

# support = []
# schoolsup = data.pop('schoolsup')
# famsup = data.pop('famsup')
# paid = data.pop('paid')

# for i in range(len(data)):
#     num_support = schoolsup[i] + famsup[i] + paid[i]
#     if num_support > 1:
#         support.append('4')  
#     elif num_support == 0:
#         support.append('0')   
#     else:
#         if schoolsup[i] == 1:
#             support.append('2')    
#         elif famsup[i] == 1:
#             support.append('1')   
#         else:
#             support.append('3') 
            
# support = pd.DataFrame({'support': support})
# data = pd.concat([data, support], axis = 1)

In [1379]:
# ########## Socializing Ratio ###########
# # goout/freetime
# goout = data.pop('goout')
# freetime = data.pop('freetime')
# social = []
# for i in range(len(freetime)):
#     if freetime[i] == 0:
#         social.append(0)
#     else:
#         social.append(goout[i]/freetime[i])
        
# social = pd.DataFrame({'Social': social})
# data = pd.concat([data, social], axis = 1)

In [1380]:
# ########## Studying Ratio ###########
# # studytime/freetime
# studytime = data.pop('studytime')
# freetime = data.pop('freetime')
# study = []
# for i in range(len(studytime)):
#     if freetime[i] == 0:
#         study.append(0)
#     else:
#         study.append(studytime[i]/freetime[i])
        
# study = pd.DataFrame({'Study': study})
# data = pd.concat([data, study], axis = 1)

In [1381]:
# ########## Repeaters ###########
# # if age > 16 and failures > 0
# age = data.pop('age')
# failures = data.pop('failures')

# Repeater = []
# for i in range(len(failures)):
#     if age[i] > 16 and failures[i] > 0:
#         Repeater.append(1)
#     else:
#         Repeater.append(0)
        
# rep = pd.DataFrame({'Repeater': Repeater})
# data = pd.concat([data, rep], axis = 1)

# Creating a Benchmark

In [1382]:
import sys
import csv
import math
import numpy as np
from operator import itemgetter
import time
import joblib

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale


#Handle annoying warnings
import warnings, sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)


In [1383]:
#############################################################################
#
# Load Data
#
#####################
       

#Read data
header = list(data.columns)
target= data.G3
data= data.iloc[: , 1: ]


#Test Print
print(header)
print(len(target),len(data))
'''for i in range(10):
    print(target[i])
    print(data[i])'''
print('\n')

data_np=np.asarray(data)
target_np=np.asarray(target)

['G3', 'school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'goout', 'health', 'age', 'absences', 'Alcohol', 'Study']
395 395




In [1384]:
#############################################################################
#
# Preprocess data
#
##########################################


if norm_target==1:
    #Target normalization for continuous values
    target_np=scale(target_np)

if norm_features==1:
    #Feature normalization for continuous values
    data_np=scale(data_np)

if binning==1:
    if prebinned == 0:
        #Discretize Target variable with KBinsDiscretizer
        enc = KBinsDiscretizer(n_bins=[bin_cnt], encode='ordinal', strategy='quantile')                         #Strategy here is important, quantile creating equal bins, but kmeans prob being more valid "clusters"
        target_np_bin = enc.fit_transform(target_np.reshape(-1,1))

        #Get Bin min/max
        temp=[[] for x in range(bin_cnt+1)]
        for i in range(len(target_np)):
            for j in range(bin_cnt):
                if target_np_bin[i]==j:
                    temp[j].append(target_np[i])

        for j in range(bin_cnt):
            print('Bin', j, ':', min(temp[j]), max(temp[j]), len(temp[j]))
        print('\n')

        #Convert Target array back to correct shape
        target_np=np.ravel(target_np_bin)

In [1385]:
#############################################################################
#
# Feature Selection
#
##########################################

#Low Variance Filter
if lv_filter==1 or feat_eng == 1:
    print('--LOW VARIANCE FILTER ON--', '\n')
    
    #LV Threshold
    sel = VarianceThreshold(threshold=0.5)                                      #Removes any feature with less than 20% variance
    fit_mod=sel.fit(data_np)
    fitted=sel.transform(data_np)
    sel_idx=fit_mod.get_support()

    #Get lists of selected and non-selected features (names and indexes)
    temp=[]
    temp_idx=[]
    temp_del=[]
    for i in range(len(data_np[0])):
        if sel_idx[i]==1:                                                           #Selected Features get added to temp header
            temp.append(header[i+feat_start])
            temp_idx.append(i)
        else:                                                                       #Indexes of non-selected features get added to delete array
            temp_del.append(i)

    print('Selected', temp)
    print('Features (total, selected):', len(data_np[0]), len(temp))
    print('\n')

    #Filter selected columns from original dataset
    if feat_eng != 1:
        header = header[0:feat_start]
        for field in temp:
            header.append(field)
        data_np = np.delete(data_np, temp_del, axis=1)                                 #Deletes non-selected features by index


#Feature Selection
if feat_select==1 or feat_eng == 1:
    '''Three steps:
       1) Run Feature Selection
       2) Get lists of selected and non-selected features
       3) Filter columns from original dataset
       '''
    
    print('--FEATURE SELECTION ON--', '\n')
    
    ##1) Run Feature Selection #######
    if fs_type==1 or feat_eng == 1:
        #Stepwise Recursive Backwards Feature removal
        if binning==1:
            clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3, criterion='entropy', random_state=None)
            sel = RFE(clf, n_features_to_select=k_cnt, step=.1)
            print('Stepwise Recursive Backwards - Random Forest: ')
        if binning==0:
            rgr = RandomForestRegressor(n_estimators=500, max_depth=None, min_samples_split=3, criterion='squared_error', random_state=None)
            sel = RFE(rgr, n_features_to_select=k_cnt, step=.1)
            print('Stepwise Recursive Backwards - Random Forest: ')
            
        fit_mod=sel.fit(data_np, target_np)
        print(sel.ranking_)
        sel_idx=fit_mod.get_support()      
        

        ##2) Get lists of selected and non-selected features (names and indexes) #######
        temp=[]
        temp_idx=[]
        temp_del=[]
        for i in range(len(data_np[0])):
            if sel_idx[i]==1:                                                           #Selected Features get added to temp header
                temp.append(header[i+feat_start])
                temp_idx.append(i)
            else:                                                                       #Indexes of non-selected features get added to delete array
                temp_del.append(i)
        print('Selected', temp)
        print('Features (total/selected):', len(data_np[0]), len(temp))
        print('\n')

                
    ##3) Filter selected columns from original dataset #########
    if feat_eng != 1:
        header = header[0:feat_start]
        for field in temp:
            header.append(field)
        data_np = np.delete(data_np, temp_del, axis=1)                                 #Deletes non-selected features by index)

    if fs_type==2 or feat_eng == 1:
        #Wrapper Select via model
        if binning==1:
            clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3, criterion='entropy', random_state=None)
            sel = SelectFromModel(clf, prefit=False, threshold='mean', max_features=None)                                                           #to select only based on max_features, set to integer value and set threshold=-np.inf
            print ('Wrapper Select - Random Forest: ')
        if binning==0:
            rgr = RandomForestRegressor(n_estimators=500, max_features=.33, max_depth=None, min_samples_split=3, criterion='squared_error', random_state=None)
            sel = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
            print ('Wrapper Select - Random Forest: ')
            
        fit_mod=sel.fit(data_np, target_np)    
        sel_idx=fit_mod.get_support()
        
        ##2) Get lists of selected and non-selected features (names and indexes) #######
        temp=[]
        temp_idx=[]
        temp_del=[]
        for i in range(len(data_np[0])):
            if sel_idx[i]==1:                                                           #Selected Features get added to temp header
                temp.append(header[i+feat_start])
                temp_idx.append(i)
            else:                                                                       #Indexes of non-selected features get added to delete array
                temp_del.append(i)
        print('Selected', temp)
        print('Features (total/selected):', len(data_np[0]), len(temp))
        print('\n')

    if fs_type==3 or feat_eng == 1:       
        if binning==1:                                                              ######Only work if the Target is binned###########
            #Univariate Feature Selection - Chi-squared
            sel=SelectKBest(chi2, k=k_cnt)
            fit_mod=sel.fit(data_np, target_np)                                         #will throw error if any negative values in features, so turn off feature normalization, or switch to mutual_info_classif
            print ('Univariate Feature Selection - Chi2: ')
            sel_idx=fit_mod.get_support()

        if binning==0:                                                              ######Only work if the Target is continuous###########
            #Univariate Feature Selection - Mutual Info Regression
            sel=SelectKBest(mutual_info_regression, k=k_cnt)
            fit_mod=sel.fit(data_np, target_np)
            print ('Univariate Feature Selection - Mutual Info: ')
            sel_idx=fit_mod.get_support()

        #Print ranked variables out sorted
        temp=[]
        scores=fit_mod.scores_
        for i in range(feat_start, len(header)):            
            temp.append([header[i], float(scores[i-feat_start])])

        print('Ranked Features')
        temp_sort=sorted(temp, key=itemgetter(1), reverse=True)                     #Doesn't always sort correctly (e.g. for Chi Sq), so doublecheck output
        for i in range(len(temp_sort)):
            print(i, temp_sort[i][0], ':', temp_sort[i][1])
        print('\n')
        
        ##2) Get lists of selected and non-selected features (names and indexes) #######
        temp=[]
        temp_idx=[]
        temp_del=[]
        for i in range(len(data_np[0])):
            if sel_idx[i]==1:                                                           #Selected Features get added to temp header
                temp.append(header[i+feat_start])
                temp_idx.append(i)
            else:                                                                       #Indexes of non-selected features get added to delete array
                temp_del.append(i)
        print('Selected', temp)
        print('Features (total/selected):', len(data_np[0]), len(temp))
        print('\n')
    
                
    ##3) Filter selected columns from original dataset #########
    if feat_eng != 1:
        header = header[0:feat_start]
        for field in temp:
            header.append(field)
        data_np = np.delete(data_np, temp_del, axis=1)                                 #Deletes non-selected features by index)




--LOW VARIANCE FILTER ON-- 

Selected ['Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'failures', 'famrel', 'goout', 'health', 'age', 'absences', 'Alcohol']
Features (total, selected): 28 12


--FEATURE SELECTION ON-- 

Stepwise Recursive Backwards - Random Forest: 
[13  6 11 10 12  2  3  4  4  5  7  7  6  8  8  9  9 11 13 12 10  5  3  1
  1  1  1  1]
Selected ['health', 'age', 'absences', 'Alcohol', 'Study']
Features (total/selected): 28 5


Wrapper Select - Random Forest: 
Selected ['Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'failures', 'famrel', 'goout', 'health', 'age', 'absences', 'Alcohol', 'Study']
Features (total/selected): 28 13


Univariate Feature Selection - Chi2: 
Ranked Features
0 absences : 99.38454293588747
1 failures : 34.69088187928619
2 Alcohol : 16.346640074935863
3 Medu : 10.772454643187384
4 schoolsup : 10.340600150767209
5 traveltime : 8.444099041201081
6 reason : 7.713171079906538
7 romantic : 4.9502176929833
8 Fedu : 3.844065596823003
9 health : 3.4035991086423096
10